# DCF Valuation Interactive Notebook

This notebook uses **ipywidgets** so you can experiment with inputs for a 5‑year explicit DCF and a terminal‑value multiple, plus an optional Monte‑Carlo simulation. Run it through **Voilà** (or simply run cell‑by‑cell in Jupyter) to hide the code and get a clean UI.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:

def compute_dcf(
    revenue0,
    growth_rates,
    ebit_margins,
    tax_rates,
    sales_capitals,
    nols,
    tv_multiple,
    tv_basis,
    debt_value,
    market_cap,
    rf_rate,
    erp,
    beta_u,
    cost_of_debt
):
    # Capital structure
    debt_cap_ratio = debt_value / (debt_value + market_cap) if (debt_value+market_cap)!=0 else 0
    beta_l = beta_u * (1 + (1 - tax_rates[0]) * debt_value / market_cap) if market_cap!=0 else beta_u
    cost_of_equity = rf_rate + beta_l * erp
    wacc = cost_of_equity * (1 - debt_cap_ratio) + cost_of_debt * debt_cap_ratio * (1 - tax_rates[0])

    years = [f"Year {i}" for i in range(1,6)]
    revenue = [revenue0]
    ebit = []
    nopat = []
    reinvestment = []
    fcff = []
    pv_fcff = []
    discount_factors = []

    for i in range(5):
        rev = revenue[-1] * (1 + growth_rates[i])
        revenue.append(rev)
        ebit_i = rev * ebit_margins[i]
        nopat_i = ebit_i * (1 - tax_rates[i])
        reinv_i = (rev - revenue[-2]) / sales_capitals[i] if sales_capitals[i]!=0 else 0
        fcff_i = nopat_i - reinv_i + nols[i]

        df = 1 / (1 + wacc)**(i+1)

        ebit.append(ebit_i)
        nopat.append(nopat_i)
        reinvestment.append(reinv_i)
        fcff.append(fcff_i)
        discount_factors.append(df)
        pv_fcff.append(fcff_i * df)

    tv_base_map = {
        "Revenue": revenue[-1],
        "EBIT": ebit[-1],
        "NOPAT": nopat[-1],
        "FCFF": fcff[-1]
    }
    terminal_value = tv_base_map[tv_basis] * tv_multiple
    pv_terminal = terminal_value / ((1 + wacc)**5)

    enterprise_value = sum(pv_fcff) + pv_terminal
    equity_value = enterprise_value - debt_value

    df_out = pd.DataFrame({
        "Revenue": revenue[1:],
        "EBIT": ebit,
        "NOPAT": nopat,
        "Reinvestment": reinvestment,
        "FCFF": fcff,
        "Discount Factor": discount_factors,
        "PV_FCFF": pv_fcff
    }, index=years)

    return df_out, pv_terminal, enterprise_value, equity_value, wacc, cost_of_equity


In [ ]:

# --- Input widgets ---
revenue0_w = widgets.FloatText(value=100.0, description='Revenue0')
debt_w = widgets.FloatText(value=50.0, description='Debt')
mcap_w = widgets.FloatText(value=150.0, description='Market Cap')

rf_w = widgets.FloatSlider(value=0.04, min=0.0, max=0.1, step=0.005, description='RF')
erp_w = widgets.FloatSlider(value=0.055, min=0.0, max=0.15, step=0.005, description='ERP')
beta_w = widgets.FloatSlider(value=1.0, min=0.0, max=3.0, step=0.05, description='Beta_u')
cod_w = widgets.FloatSlider(value=0.05, min=0.0, max=0.2, step=0.005, description='CostDebt')

# 5x sliders for growth, margins etc
growth_ws = [widgets.FloatSlider(description=f'g{i+1} (%)', value=10.0, min=-20, max=40, step=1) for i in range(5)]
margin_ws = [widgets.FloatSlider(description=f'EBIT%{i+1}', value=20.0, min=-50, max=50, step=1) for i in range(5)]
tax_ws    = [widgets.FloatSlider(description=f'Tax%{i+1}', value=25.0, min=0, max=60, step=1) for i in range(5)]
sc_ws     = [widgets.FloatSlider(description=f'S/C{i+1}', value=2.0, min=0.1, max=10, step=0.1) for i in range(5)]
nol_ws    = [widgets.FloatText(description=f'NOL{i+1}', value=0.0) for i in range(5)]

tv_mult_w = widgets.FloatText(value=10.0, description='TV Multiple')
tv_basis_w = widgets.Dropdown(options=['Revenue','EBIT','NOPAT','FCFF'], value='EBIT', description='TV basis')

run_button = widgets.Button(description='Run DCF', button_style='success')
out = widgets.Output()

# Layout
left_box = widgets.VBox([revenue0_w, debt_w, mcap_w, rf_w, erp_w, beta_w, cod_w, tv_mult_w, tv_basis_w, run_button])
grid_box = widgets.VBox([widgets.HTML('<b>Assumptions per Year</b>')] +
                        [widgets.HBox([growth_ws[i], margin_ws[i], tax_ws[i], sc_ws[i], nol_ws[i]]) for i in range(5)])

display(widgets.HBox([left_box, grid_box]))
display(out)

# Callback
def on_run_clicked(b):
    with out:
        clear_output()
        gr = np.array([w.value/100 for w in growth_ws])
        mar = np.array([w.value/100 for w in margin_ws])
        tax = np.array([w.value/100 for w in tax_ws])
        sc = np.array([w.value      for w in sc_ws])
        nol = np.array([w.value     for w in nol_ws])

        df_res, pv_t, ev, eqv, wacc, coe = compute_dcf(
            revenue0_w.value, gr, mar, tax, sc, nol,
            tv_mult_w.value, tv_basis_w.value,
            debt_w.value, mcap_w.value,
            rf_w.value, erp_w.value, beta_w.value, cod_w.value
        )
        display(df_res.style.format("{:,.2f}"))
        print(f"PV( Terminal ): {pv_t:,.2f}")
        print(f"Enterprise Value: {ev:,.2f}")
        print(f"Equity Value: {eqv:,.2f}")
        print(f"WACC: {wacc*100:.2f}%   Cost of Equity: {coe*100:.2f}%")

run_button.on_click(on_run_clicked)


In [ ]:

# Monte Carlo simulation (optional)
import scipy.stats as stats

def triangular_correlated(low, mode, high, corr_matrix, n):
    dim = len(low)
    mean = np.zeros(dim)
    L = np.linalg.cholesky(corr_matrix)
    rnd = np.dot(np.random.randn(n, dim), L.T)
    u = stats.norm.cdf(rnd)  # uniform (0,1)
    samples = np.zeros_like(u)
    for i in range(dim):
        c = (mode[i] - low[i]) / (high[i] - low[i])
        samples[:, i] = stats.triang.ppf(u[:, i], c, loc=low[i], scale=high[i]-low[i])
    return samples

def run_monte_carlo(n_sims=1000):
    # Pack widget values
    low = np.array([5, 15, 20, 8, 6], dtype=float)
    mode= np.array([10,20, 25,10,10], dtype=float)
    high= np.array([15,25, 30,12,14], dtype=float)
    corr = np.identity(5)
    draws = triangular_correlated(low, mode, high, corr, n_sims)

    eq_vals = []
    for g, m, t, coe, mult in draws:
        g/=100; m/=100; t/=100
        rev1 = revenue0_w.value * (1+g)
        ebit1 = rev1*m
        nopat1 = ebit1*(1-t)
        reinv1 = (rev1 - revenue0_w.value) / sc_ws[0].value
        fcff1 = nopat1 - reinv1
        wacc_sim = coe/100
        ev = fcff1/wacc_sim + mult*(ebit1 if tv_basis_w.value=='EBIT' else nopat1)
        eq_vals.append(ev - debt_w.value)
    return np.array(eq_vals)

# Demo run button
mc_button = widgets.Button(description='Run Monte Carlo', button_style='info')
mc_out = widgets.Output()
display(mc_button, mc_out)

def on_mc(b):
    with mc_out:
        clear_output()
        vals = run_monte_carlo(1000)
        pctls = np.percentile(vals, [1,10,25,50,75,90,99])
        print("Percentiles:")
        for p,v in zip([1,10,25,50,75,90,99], pctls):
            print(f"{p}th: {v:,.2f}")
        plt.hist(vals, bins=30)
        plt.xlabel("Equity Value")
        plt.ylabel("Freq")
        plt.show()
mc_button.on_click(on_mc)
